### Imports

In [17]:
import json
import os

import pandas as pd

from utils.utils_go import *

# %load_ext autotime

### Parameters

In [18]:
dict_dataset = {
    0: ["mutant_a_format", "WT"],
    1: ["reinhard_format", "FCSglc"],
    2: ["hojas_format", "Nueva"],
    3: ["tea_format", "Herbal"],
    4: ["reinhard_new_1_format", "WT-YNB"],
    5: ["reinhard_new_2_format", "WT-YNB"]
}
dataset = dict_dataset[5] # change
dataset

['reinhard_new_2_format', 'WT-YNB']

In [19]:
params = {
    "methods": ["dgi-tran", "argva-base", "vgae-line", "vgae-base"], # ["deep-walk", "node2vec15", "node2vec12", "dgi-indu", "dgi-tran", "argva-base", "vgae-line", "vgae-base"], # ["vgae", "dgi", "dgi-tran", "dgi-indu", "argva-base", "vgae-line", "vgae-base"]
    "data_variations": ["none", "str", "dyn"],
    "has_transformation": False, # True or False # change
    "control": dataset[1] ,
    "dimension": 3,
    "threshold_corr": 0.3,
    "threshold_log2": 0,
    "alpha": 0.05,
    "iterations": 1,
    "raw_data_file": dataset[0] ,
    "obs": "",
    "seeds": [41, 42, 43, 44, 45, 46]
}

In [20]:
dir_path = "output"

res = sorted(os.listdir(dir_path))
n = len(res)
exp = "exp{}".format(n)
exp

'exp65'

### Load dataset

In [21]:
# load dataset groups
df_raw = pd.read_csv("raw_data/{}.csv".format(params["raw_data_file"]), delimiter="|")
# df_raw = pd.read_excel("raw_data/{}.xlsx".format(params["raw_data_file"]))
df_raw

,Alignment ID,Average Mz,Metabolite name,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
0,0,0,1-Methylnicotinamide,6.508331e+04,5.510899e+04,6.418729e+04,5.865636e+04,7.023975e+04,6.072488e+04,6.418729e+04,...,1.174696e+04,1.207721e+04,1.276583e+04,1.096030e+04,1.233098e+04,1.096030e+04,1.301666e+05,1.197777e+05,1.454335e+05,1.197777e+05
1,1,1,2-Hydroxyglutarate,3.780118e+06,3.702325e+06,6.536106e+06,6.313462e+06,3.995653e+06,3.601084e+06,3.268053e+06,...,8.928584e+06,1.689391e+07,1.282031e+06,1.813066e+06,4.023445e+06,1.813066e+06,1.171560e+06,9.516028e+05,1.005861e+06,9.516028e+05
2,2,2,2-Methylglutaric acid,7.508014e+06,6.226543e+06,5.572912e+06,6.861060e+06,6.269852e+06,5.931642e+06,6.056278e+06,...,2.097152e+06,2.294900e+06,2.767209e+06,2.617939e+06,2.511295e+06,2.617939e+06,4.564192e+05,5.029307e+05,4.692509e+05,5.029307e+05
3,3,3,3-Hydroxymethylglutarate,2.294900e+06,2.186209e+06,2.294900e+06,2.082666e+06,2.111739e+06,2.068280e+06,1.970322e+06,...,3.891589e+05,5.503539e+05,5.390277e+05,6.191803e+05,6.321906e+05,6.191803e+05,5.465523e+05,4.288162e+05,4.857990e+05,4.288162e+05
4,4,4,3-Phosphoglycerate,4.439384e+05,3.435118e+05,4.408719e+05,5.029307e+05,3.411390e+05,4.170902e+05,3.891589e+05,...,9.128384e+05,1.432397e+06,1.301666e+05,1.569559e+05,4.932393e+04,1.569559e+05,2.301042e+04,2.432243e+04,2.059491e+04,2.432243e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,107,107,Uridine monophosphate,1.975596e+04,3.511987e+04,4.445321e+04,2.994443e+04,2.643204e+04,3.322542e+04,2.832916e+04,...,5.825119e+04,8.829230e+04,9.255812e+05,1.131653e+06,9.128384e+05,1.131653e+06,1.103654e+04,1.051382e+04,1.015569e+04,1.051382e+04
108,108,108,Uridine triphosphate,2.957167e+03,5.042768e+03,3.350127e+03,4.299640e+03,3.983995e+03,1.396957e+04,1.029745e+04,...,2.102765e+04,6.330360e+04,5.397486e+04,2.793914e+04,1.895118e+04,2.793914e+04,3.104188e+03,2.936740e+03,3.213656e+03,2.936740e+03
109,109,109,Valine,3.041062e+08,2.665812e+08,2.135502e+08,2.703026e+08,3.591472e+08,3.148305e+08,3.743987e+08,...,8.435674e+07,8.978680e+07,1.448516e+08,1.296458e+08,1.296458e+08,1.296458e+08,1.541757e+08,1.370379e+08,1.428573e+08,1.370379e+08
110,110,110,Xanthine,1.240017e+05,1.021267e+05,1.301666e+05,9.528754e+04,1.189504e+05,9.268190e+04,1.000249e+05,...,2.483350e+04,2.333164e+04,2.266332e+05,3.507297e+05,2.549757e+05,3.507297e+05,2.071046e+05,1.042726e+05,1.869027e+04,1.042726e+05


### Format dataset

In [22]:
# has transformation
columns_data = list(df_raw.columns)[3:]
if params["has_transformation"]:
    print("transformation")
    for column in columns_data:
        df_raw[column] = df_raw[column].apply(lambda x: 10**x)
df_raw

,Alignment ID,Average Mz,Metabolite name,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
0,0,0,1-Methylnicotinamide,6.508331e+04,5.510899e+04,6.418729e+04,5.865636e+04,7.023975e+04,6.072488e+04,6.418729e+04,...,1.174696e+04,1.207721e+04,1.276583e+04,1.096030e+04,1.233098e+04,1.096030e+04,1.301666e+05,1.197777e+05,1.454335e+05,1.197777e+05
1,1,1,2-Hydroxyglutarate,3.780118e+06,3.702325e+06,6.536106e+06,6.313462e+06,3.995653e+06,3.601084e+06,3.268053e+06,...,8.928584e+06,1.689391e+07,1.282031e+06,1.813066e+06,4.023445e+06,1.813066e+06,1.171560e+06,9.516028e+05,1.005861e+06,9.516028e+05
2,2,2,2-Methylglutaric acid,7.508014e+06,6.226543e+06,5.572912e+06,6.861060e+06,6.269852e+06,5.931642e+06,6.056278e+06,...,2.097152e+06,2.294900e+06,2.767209e+06,2.617939e+06,2.511295e+06,2.617939e+06,4.564192e+05,5.029307e+05,4.692509e+05,5.029307e+05
3,3,3,3-Hydroxymethylglutarate,2.294900e+06,2.186209e+06,2.294900e+06,2.082666e+06,2.111739e+06,2.068280e+06,1.970322e+06,...,3.891589e+05,5.503539e+05,5.390277e+05,6.191803e+05,6.321906e+05,6.191803e+05,5.465523e+05,4.288162e+05,4.857990e+05,4.288162e+05
4,4,4,3-Phosphoglycerate,4.439384e+05,3.435118e+05,4.408719e+05,5.029307e+05,3.411390e+05,4.170902e+05,3.891589e+05,...,9.128384e+05,1.432397e+06,1.301666e+05,1.569559e+05,4.932393e+04,1.569559e+05,2.301042e+04,2.432243e+04,2.059491e+04,2.432243e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,107,107,Uridine monophosphate,1.975596e+04,3.511987e+04,4.445321e+04,2.994443e+04,2.643204e+04,3.322542e+04,2.832916e+04,...,5.825119e+04,8.829230e+04,9.255812e+05,1.131653e+06,9.128384e+05,1.131653e+06,1.103654e+04,1.051382e+04,1.015569e+04,1.051382e+04
108,108,108,Uridine triphosphate,2.957167e+03,5.042768e+03,3.350127e+03,4.299640e+03,3.983995e+03,1.396957e+04,1.029745e+04,...,2.102765e+04,6.330360e+04,5.397486e+04,2.793914e+04,1.895118e+04,2.793914e+04,3.104188e+03,2.936740e+03,3.213656e+03,2.936740e+03
109,109,109,Valine,3.041062e+08,2.665812e+08,2.135502e+08,2.703026e+08,3.591472e+08,3.148305e+08,3.743987e+08,...,8.435674e+07,8.978680e+07,1.448516e+08,1.296458e+08,1.296458e+08,1.296458e+08,1.541757e+08,1.370379e+08,1.428573e+08,1.370379e+08
110,110,110,Xanthine,1.240017e+05,1.021267e+05,1.301666e+05,9.528754e+04,1.189504e+05,9.268190e+04,1.000249e+05,...,2.483350e+04,2.333164e+04,2.266332e+05,3.507297e+05,2.549757e+05,3.507297e+05,2.071046e+05,1.042726e+05,1.869027e+04,1.042726e+05


In [23]:
# concat
df_join_raw = pd.concat([
    df_raw.iloc[:, :]], axis=1)
df_join_raw.set_index("Alignment ID", inplace=True)
df_join_raw

,Average Mz,Metabolite name,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,S51A-YNB_2.2,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
Alignment ID,,,,,,,,,,,,,,,,,,,,,
0,0,1-Methylnicotinamide,6.508331e+04,5.510899e+04,6.418729e+04,5.865636e+04,7.023975e+04,6.072488e+04,6.418729e+04,7.221445e+04,...,1.174696e+04,1.207721e+04,1.276583e+04,1.096030e+04,1.233098e+04,1.096030e+04,1.301666e+05,1.197777e+05,1.454335e+05,1.197777e+05
1,1,2-Hydroxyglutarate,3.780118e+06,3.702325e+06,6.536106e+06,6.313462e+06,3.995653e+06,3.601084e+06,3.268053e+06,3.478418e+06,...,8.928584e+06,1.689391e+07,1.282031e+06,1.813066e+06,4.023445e+06,1.813066e+06,1.171560e+06,9.516028e+05,1.005861e+06,9.516028e+05
2,2,2-Methylglutaric acid,7.508014e+06,6.226543e+06,5.572912e+06,6.861060e+06,6.269852e+06,5.931642e+06,6.056278e+06,6.627347e+06,...,2.097152e+06,2.294900e+06,2.767209e+06,2.617939e+06,2.511295e+06,2.617939e+06,4.564192e+05,5.029307e+05,4.692509e+05,5.029307e+05
3,3,3-Hydroxymethylglutarate,2.294900e+06,2.186209e+06,2.294900e+06,2.082666e+06,2.111739e+06,2.068280e+06,1.970322e+06,1.997827e+06,...,3.891589e+05,5.503539e+05,5.390277e+05,6.191803e+05,6.321906e+05,6.191803e+05,5.465523e+05,4.288162e+05,4.857990e+05,4.288162e+05
4,4,3-Phosphoglycerate,4.439384e+05,3.435118e+05,4.408719e+05,5.029307e+05,3.411390e+05,4.170902e+05,3.891589e+05,4.000997e+05,...,9.128384e+05,1.432397e+06,1.301666e+05,1.569559e+05,4.932393e+04,1.569559e+05,2.301042e+04,2.432243e+04,2.059491e+04,2.432243e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,107,Uridine monophosphate,1.975596e+04,3.511987e+04,4.445321e+04,2.994443e+04,2.643204e+04,3.322542e+04,2.832916e+04,2.317048e+04,...,5.825119e+04,8.829230e+04,9.255812e+05,1.131653e+06,9.128384e+05,1.131653e+06,1.103654e+04,1.051382e+04,1.015569e+04,1.051382e+04
108,108,Uridine triphosphate,2.957167e+03,5.042768e+03,3.350127e+03,4.299640e+03,3.983995e+03,1.396957e+04,1.029745e+04,1.199379e+04,...,2.102765e+04,6.330360e+04,5.397486e+04,2.793914e+04,1.895118e+04,2.793914e+04,3.104188e+03,2.936740e+03,3.213656e+03,2.936740e+03
109,109,Valine,3.041062e+08,2.665812e+08,2.135502e+08,2.703026e+08,3.591472e+08,3.148305e+08,3.743987e+08,3.350962e+08,...,8.435674e+07,8.978680e+07,1.448516e+08,1.296458e+08,1.296458e+08,1.296458e+08,1.541757e+08,1.370379e+08,1.428573e+08,1.370379e+08


In [24]:
# split
df_join_raw = df_join_raw.rename_axis(None)
# df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

,Average Mz,Metabolite name,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,S51A-YNB_2.2,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
0,0,1-Methylnicotinamide,6.508331e+04,5.510899e+04,6.418729e+04,5.865636e+04,7.023975e+04,6.072488e+04,6.418729e+04,7.221445e+04,...,1.174696e+04,1.207721e+04,1.276583e+04,1.096030e+04,1.233098e+04,1.096030e+04,1.301666e+05,1.197777e+05,1.454335e+05,1.197777e+05
1,1,2-Hydroxyglutarate,3.780118e+06,3.702325e+06,6.536106e+06,6.313462e+06,3.995653e+06,3.601084e+06,3.268053e+06,3.478418e+06,...,8.928584e+06,1.689391e+07,1.282031e+06,1.813066e+06,4.023445e+06,1.813066e+06,1.171560e+06,9.516028e+05,1.005861e+06,9.516028e+05
2,2,2-Methylglutaric acid,7.508014e+06,6.226543e+06,5.572912e+06,6.861060e+06,6.269852e+06,5.931642e+06,6.056278e+06,6.627347e+06,...,2.097152e+06,2.294900e+06,2.767209e+06,2.617939e+06,2.511295e+06,2.617939e+06,4.564192e+05,5.029307e+05,4.692509e+05,5.029307e+05
3,3,3-Hydroxymethylglutarate,2.294900e+06,2.186209e+06,2.294900e+06,2.082666e+06,2.111739e+06,2.068280e+06,1.970322e+06,1.997827e+06,...,3.891589e+05,5.503539e+05,5.390277e+05,6.191803e+05,6.321906e+05,6.191803e+05,5.465523e+05,4.288162e+05,4.857990e+05,4.288162e+05
4,4,3-Phosphoglycerate,4.439384e+05,3.435118e+05,4.408719e+05,5.029307e+05,3.411390e+05,4.170902e+05,3.891589e+05,4.000997e+05,...,9.128384e+05,1.432397e+06,1.301666e+05,1.569559e+05,4.932393e+04,1.569559e+05,2.301042e+04,2.432243e+04,2.059491e+04,2.432243e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,107,Uridine monophosphate,1.975596e+04,3.511987e+04,4.445321e+04,2.994443e+04,2.643204e+04,3.322542e+04,2.832916e+04,2.317048e+04,...,5.825119e+04,8.829230e+04,9.255812e+05,1.131653e+06,9.128384e+05,1.131653e+06,1.103654e+04,1.051382e+04,1.015569e+04,1.051382e+04
108,108,Uridine triphosphate,2.957167e+03,5.042768e+03,3.350127e+03,4.299640e+03,3.983995e+03,1.396957e+04,1.029745e+04,1.199379e+04,...,2.102765e+04,6.330360e+04,5.397486e+04,2.793914e+04,1.895118e+04,2.793914e+04,3.104188e+03,2.936740e+03,3.213656e+03,2.936740e+03
109,109,Valine,3.041062e+08,2.665812e+08,2.135502e+08,2.703026e+08,3.591472e+08,3.148305e+08,3.743987e+08,3.350962e+08,...,8.435674e+07,8.978680e+07,1.448516e+08,1.296458e+08,1.296458e+08,1.296458e+08,1.541757e+08,1.370379e+08,1.428573e+08,1.370379e+08
110,110,Xanthine,1.240017e+05,1.021267e+05,1.301666e+05,9.528754e+04,1.189504e+05,9.268190e+04,1.000249e+05,9.661771e+04,...,2.483350e+04,2.333164e+04,2.266332e+05,3.507297e+05,2.549757e+05,3.507297e+05,2.071046e+05,1.042726e+05,1.869027e+04,1.042726e+05


In [25]:
# get groups name
groups_id = []
for item in df_join_raw.iloc[:, 2:].columns.values:
    group_id = item.split("_")[0]
    if group_id not in groups_id:
        groups_id.append(group_id)
groups_id

['WT-YNB',
 'S51A-YNB',
 'WT-Tyr',
 'S51A-Tyr',
 'WT-total',
 'WT-vac',
 'avt1-total',
 'avt1-vac',
 'aa-standard']

In [26]:
# get subgroups names
subgroups_id = get_subgroups_id(df_join_raw, groups_id)
subgroups_id

{'WT-YNB': ['1', '2'],
 'S51A-YNB': ['1', '2'],
 'WT-Tyr': ['1', '2'],
 'S51A-Tyr': ['1', '2'],
 'WT-total': ['1', '2'],
 'WT-vac': ['1', '2'],
 'avt1-total': ['1', '2'],
 'avt1-vac': ['1', '2'],
 'aa-standard': ['1', '2']}

In [27]:
# get groups combination
groups = []
control = params["control"]

groups = []
properties = list(subgroups_id.keys())
properties.remove(control)
for group in properties:
    aux = [control, group]
    groups.append(aux)

### Create folders

In [28]:
# create experiments folder
try: 
    os.mkdir("output/{}".format(exp))
    os.mkdir("output/{}/correlations".format(exp))
    os.mkdir("output/{}/preprocessing".format(exp))
    os.mkdir("output/{}/preprocessing/edges".format(exp))
    os.mkdir("output/{}/preprocessing/graphs".format(exp))
    os.mkdir("output/{}/preprocessing/graphs_data".format(exp))
    os.mkdir("output/{}/node_embeddings".format(exp))
    os.mkdir("output/{}/edge_embeddings".format(exp))
    os.mkdir("output/{}/common_edges".format(exp))
    os.mkdir("output/{}/changes".format(exp))
    os.mkdir("output/{}/plots".format(exp))
    os.mkdir("output/{}/biocyc".format(exp))
except OSError as error: 
    print(error)

### Save dataset and parameters

In [29]:
# save dataset
df_join_raw.to_csv("input/{}_raw.csv".format(exp), index=True)

# save parameters
parameters = {
    "exp": exp,
    "methods": params["methods"],
    "data_variations": params["data_variations"],
    "has_transformation": params["has_transformation"],
    "control": params["control"],
    "dimension": params["dimension"],
    "threshold_corr": params["threshold_corr"],
    "threshold_log2": params["threshold_log2"],
    "alpha": params["alpha"],
    "iterations": params["iterations"],
    
    "groups_id": groups_id,
    "subgroups_id": subgroups_id,
    "groups": groups,

    "seeds": params["seeds"],
    "obs": params["obs"]
}

with open("output/{}/parameters.json".format(exp), "w") as outfile:
    json.dump(parameters, outfile, indent=4)

In [30]:
experiments = {
    "exp": exp
}

with open("exp.json".format(experiments), "w") as outfile:
    json.dump(experiments, outfile, indent=4)